In [5]:
import numpy as np
import pandas as pd
import os
import chardet

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
%matplotlib inline

## Process Data

### 1. Read in data

In [8]:
application_train = pd.read_csv('data/application_train.csv', index_col=0)
print('The dimension of the training data: ')
print(application_train.shape)
application_test = pd.read_csv('data/application_test.csv', index_col=0)
print('The dimension of the testing data: ')
print(application_test.shape)

The dimension of the training data: 
(307511, 121)
The dimension of the testing data: 
(48744, 120)


### 2. Show examples of training testing data

In [10]:
application_train.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
application_test.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
os.listdir('data/')

['application_test.csv',
 'application_train.csv',
 'bureau.csv',
 'bureau_balance.csv',
 'credit_card_balance.csv',
 'HomeCredit_columns_description.csv',
 'installments_payments.csv',
 'POS_CASH_balance.csv',
 'previous_application.csv',
 'sample_submission.csv']

### 3. Concat two original dataset together for processing

In [13]:
full_set = pd.concat([application_train, application_test], sort=False)
print('The shape of the concatanated dataset is: ')
print(full_set.shape)

The shape of the concatanated dataset is: 
(356255, 121)


In [14]:
full_set.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


### 4. Get dummy variables

In [15]:
full_set_with_dummies = pd.get_dummies(full_set, drop_first=True)

In [16]:
full_set_with_dummies.shape

(356255, 229)

In [17]:
full_set_with_dummies.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,...,0,0,0,0,0,0,0,1,0,0
100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,...,0,0,0,0,0,0,0,0,0,0
100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,...,0,0,0,0,0,0,0,0,0,0
100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,...,0,0,0,0,0,0,0,0,0,0
100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,...,0,0,0,0,0,0,0,0,0,0


### 5. Construct the training set and labels

In [19]:
train_test_cols = list(full_set_with_dummies.columns)
train_test_cols.remove('TARGET')

In [20]:
X = full_set_with_dummies.loc[full_set_with_dummies['TARGET'].notna(), train_test_cols]
y = full_set_with_dummies.loc[full_set_with_dummies['TARGET'].notna(), 'TARGET']
test = full_set_with_dummies.loc[full_set_with_dummies['TARGET'].isna(), train_test_cols]

In [21]:
print(X.shape)
print(y.shape)
print(test.shape)

(307511, 228)
(307511,)
(48744, 228)


### 6. Construct a balanced dataset with equal number of each label

In [22]:
# Try another model with balanced dataset
label_1 = full_set_with_dummies.loc[full_set_with_dummies['TARGET'] == 1.0]
label_0 = full_set_with_dummies.loc[full_set_with_dummies['TARGET'] == 0]
label_0_samples = label_0.sample(label_1.shape[0])
print(label_0_samples.shape)

(24825, 229)


In [23]:
balanced = pd.concat([label_1, label_0_samples], sort=False)
X = balanced.loc[:,train_test_cols]
y = balanced.loc[:,'TARGET']

In [24]:
print(X.shape)
print(y.shape)

(49650, 228)
(49650,)


### 7. Impute the missing values

In [31]:
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
X_imp = imp.fit_transform(X)
test_imp = imp.fit_transform(test)

# Modeling

In [41]:
def rf(X, y, n_estimators=500):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    X_train_imp = imp.fit_transform(X_train)
    X_test_imp = imp.fit_transform(X_test)
    clf = RandomForestClassifier(n_estimators=n_estimators)
    clf.fit(X_train_imp, y_train)
    y_pred_rt = clf.predict_proba(X_test_imp)[:, 1]
    plot_roc(y_test, y_pred_rt)
    return clf

def lr(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    X_train_imp = imp.fit_transform(X_train)
    X_test_imp = imp.fit_transform(X_test)
    clf = RandomForestClassifier(n_estimators=n_estimators)
    clf.fit(X_train_imp, y_train)
    y_pred_rt = clf.predict_proba(X_test_imp)[:, 1]
    fpr_rt, tpr_rt, _ = roc_curve(y_test, y_pred_rt)

    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr_rt, tpr_rt, label='RF')
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.show()
    return clf

def predict(clf, X, y, test):
    clf.fit(X, y)
    return clf.predict(test)

def create_submission(file_location, y_pred):
    submissions = pd.read_csv('data/sample_submission.csv')
    submissions.loc[:,'TARGET'] = y_pred
    submissions.to_csv(file_location, index = False)
    
def plot_roc(y_test, y_pred_rt):
    fpr_rt, tpr_rt, _ = roc_curve(y_test, y_pred_rt)
    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr_rt, tpr_rt, label='RF')
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.show()

### 1. Random Forest
#### a. mean inputation
#### b. 500 trees

In [ ]:
clf = rf(X, y)
y_pred = predict(clf, X_imp, y, test_imp)
len(y_pred)
create_submission('submissions/RF_500.csv', y_pred)